# Lab | Data Aggregation and Filtering

In [2]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

In [4]:
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [16]:
df.columns = [col.replace(" ", "_").lower() for col in df.columns]

In [20]:
df_filtered = df[df['total_claim_amount'] > 1000]

In [22]:
df_filtered_yes = df_filtered[df_filtered['response'] == 'Yes']

In [24]:
df_filtered_yes.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


In [32]:
average_claims = df[df['response'] == 'Yes'].groupby(['policy_type', 'gender'])['total_claim_amount'].mean()

In [34]:
average_claims

policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64

In [44]:
customer_count_by_state = df['state'].value_counts()
customer_count_by_state

state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

In [48]:
state_with_more_than_500 = customer_count_by_state[customer_count_by_state > 500]
state_with_more_than_500

state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

In [58]:
stats_by_life_educ = df.groupby(['education', 'gender']).agg({"customer_lifetime_value": ["max", "min", "median"]})

In [60]:
stats_by_life_educ

customer_lifetime_value                          
                                                max          min       median
education            gender                                                  
Bachelor             F                  73225.95652  1904.000852  5640.505303
                     M                  67907.27050  1898.007675  5548.031892
College              F                  61850.18803  1898.683686  5623.611187
                     M                  61134.68307  1918.119700  6005.847375
Doctor               F                  44856.11397  2395.570000  5332.462694
                     M                  32677.34284  2267.604038  5577.669457
High School or Below F                  55277.44589  2144.921535  6039.553187
                     M                  83325.38119  1940.981221  6286.731006
Master               F                  51016.06704  2417.777032  5729.855012
                     M                  50568.25912  2272.307310  5579.099207

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [66]:
#Question 5
policies_state_by_month = df.pivot_table(index= 'state', columns = 'effective_to_date', values = 'number_of_policies', aggfunc='sum')

policies_state_by_month

effective_to_date,1/1/11,1/10/11,1/11/11,1/12/11,1/13/11,1/14/11,1/15/11,1/16/11,1/17/11,1/18/11,...,2/26/11,2/27/11,2/28/11,2/3/11,2/4/11,2/5/11,2/6/11,2/7/11,2/8/11,2/9/11
state,,,,,,,,,,,,,,,,,,,,,
Arizona,139,123,90,101,115,97,97,74,124,128,...,103,110,90,137,157,133,93,93,119,92
California,180,225,270,155,179,145,165,184,162,241,...,199,193,135,232,165,202,180,238,108,168
Nevada,36,41,37,30,59,23,45,15,37,61,...,29,60,60,62,42,14,31,40,56,32
Oregon,111,215,130,92,110,143,125,133,210,172,...,145,175,168,123,173,152,145,171,102,105
Washington,84,75,23,36,47,50,33,28,40,46,...,50,48,49,38,46,55,46,32,33,35


In [ ]:
#Question 6

# Convert 'effective_to_date' column to datetime format
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# Grouping the data by state and month, and calculating the sum of the number_of_policies for each group
policies_by_state_month = df.groupby(['state', df['effective_to_date'].dt.month])['number_of_policies'].sum().reset_index()

# Sorting the data by the count of policies sold in descending order
policies_by_state_month_sorted = policies_by_state_month.sort_values(by='number_of_policies', ascending=False)

# Selecting the top 3 states with the highest number of policies sold
top_3_states = policies_by_state_month_sorted['state'].head(3)

# Creating a new DataFrame that contains the number of policies sold by month for each of the top 3 states
new_df = policies_by_state_month_sorted[policies_by_state_month_sorted['state'].isin(top_3_states)]

In [72]:
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%Y-%m-%d')

In [76]:
policies_by_state_month = df.groupby(['state', df['effective_to_date'].dt.month])['number_of_policies'].sum().reset_index()
policies_by_state_month

,state,effective_to_date,number_of_policies
0,Arizona,1,3052
1,Arizona,2,2864
2,California,1,5673
3,California,2,4929
4,Nevada,1,1493
5,Nevada,2,1278
6,Oregon,1,4697
7,Oregon,2,3969
8,Washington,1,1358
9,Washington,2,1225


In [78]:
policies_by_state_month_sorted = policies_by_state_month.sort_values(by='number_of_policies', ascending=False)
policies_by_state_month_sorted

,state,effective_to_date,number_of_policies
2,California,1,5673
3,California,2,4929
6,Oregon,1,4697
7,Oregon,2,3969
0,Arizona,1,3052
1,Arizona,2,2864
4,Nevada,1,1493
8,Washington,1,1358
5,Nevada,2,1278
9,Washington,2,1225


In [80]:
top_3_states = policies_by_state_month_sorted['state'].head(3)
top_3_states

2    California
3    California
6        Oregon
Name: state, dtype: object

In [82]:
new_df = policies_by_state_month_sorted[policies_by_state_month_sorted['state'].isin(top_3_states)]
new_df

,state,effective_to_date,number_of_policies
2,California,1,5673
3,California,2,4929
6,Oregon,1,4697
7,Oregon,2,3969
